In [9]:
import pandas as pd
import numpy as np
import os
import sys

# --- 0. 경로 설정 ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(PROJECT_ROOT, "data", "processed")
FINAL_MASTER_FILE = os.path.join(DATA_DIR, "final_master_table_v2.csv")


## Phase2-A : Data Handler 정의

12개 채널의 Scale 일치를 위해 Standardization을 진행합니다.
<br>모든 12개 채널의 Scale을평균 0, 표준편차 1인 표준 정규 분포로 통일시킵니다.

In [10]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler

# [1] --- DataHandler 클래스 정의 (V2: Scaler + zfill) ---
# (Phase 2-A와 2-C가 합쳐진 최종 버전)
import pandas as pd
import numpy as np
import os
import sys

# --- 0. 경로 설정 ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(PROJECT_ROOT, "data", "processed")
FINAL_MASTER_FILE = os.path.join(DATA_DIR, "final_master_table_v2.csv")

class DataHandler:
    """
    [V2] 표준화(Standardization)와 zfill(6)이 적용된 DataHandler.
    """
    
    def __init__(self, file_path, train_end_date='2022-12-31'):
        self.file_path = file_path
        self.train_end_date = pd.to_datetime(train_end_date)
        self.data_by_ticker = {}   # 원본 데이터
        self.scalers_by_ticker = {} # Ticker별 Scaler
        self.tickers = []
        
        self._load_and_process_data()
        self._fit_scalers()
        
    def _load_and_process_data(self):
        try:
            # 1. dtype=str로 읽기
            df = pd.read_csv(
                self.file_path, 
                parse_dates=['date'],
                dtype={'ticker': str} 
            )
            # 2. zfill(6)로 '0' 채우기
            df['ticker'] = df['ticker'].str.zfill(6)
            df = df.set_index('date')
            
            self.tickers = df['ticker'].unique()
            
            for ticker in self.tickers:
                ticker_df = df[df['ticker'] == ticker].copy()
                channel_cols = [col for col in ticker_df.columns if col not in ['ticker']]
                self.data_by_ticker[ticker] = ticker_df[channel_cols]
            
            print(f"[DataHandler V2] Success: Loaded {len(self.tickers)} tickers.")
            print(f"[DataHandler V2] Available tickers: {self.tickers}")

        except Exception as e:
            print(f"[DataHandler V2] Error loading data: {e}")

    def _fit_scalers(self):
        """
        [Data Leakage 방지] 훈련 데이터로만 Scaler를 학습(fit)
        """
        print(f"[DataHandler V2] Fitting scalers using data up to {self.train_end_date.date()}...")
        for ticker in self.tickers:
            train_data = self.data_by_ticker[ticker].loc[:self.train_end_date]
            if train_data.empty:
                print(f"  > Warning: No training data for {ticker}.")
                continue
            
            scaler = StandardScaler()
            scaler.fit(train_data) # 'fit'은 훈련 데이터로만!
            self.scalers_by_ticker[ticker] = scaler
        print("[DataHandler V2] Scalers fitted.")

    def get_scaled_data_by_ticker(self, ticker):
        """
        'transform'은 전체 데이터에 적용하여 표준화된 DF 반환
        """
        if ticker not in self.scalers_by_ticker:
            print(f"[DataHandler V2] Error: No scaler for {ticker}")
            return None
        
        original_data = self.data_by_ticker[ticker]
        scaler = self.scalers_by_ticker[ticker]
        
        scaled_data_np = scaler.transform(original_data)
        
        scaled_df = pd.DataFrame(
            scaled_data_np, 
            index=original_data.index, 
            columns=original_data.columns
        )
        return scaled_df

    def get_all_tickers(self):
        return self.tickers


## Phase2-B : 윈도우 생성기 구현
LLM에 넣기 위해 슬라이딩 윈도우 방식의 3D 데이터로 재구성합니다.
<br>(샘플 수, 120일, 12개 채널) 형태의 numpy 배열(텐서)을 만듭니다.

In [11]:
# [2] --- 윈도우 생성기 함수 정의 ---
# (Phase 2-B)

def create_sliding_windows(data, input_seq_len, output_seq_len):
    """
    DataFrame(2D)을 슬라이딩 윈도우(3D) numpy 배열로 변환합니다.
    """
    data_np = data.values
    n_samples = len(data_np)
    X, y = [], []
    
    total_len = input_seq_len + output_seq_len
    
    for i in range(n_samples - total_len + 1):
        input_window = data_np[i : i + input_seq_len]
        output_window = data_np[i + input_seq_len : i + total_len]
        X.append(input_window)
        y.append(output_window)
        
    return np.array(X), np.array(y)

# --- 3. Phase 2 파이프라인 실행 테스트 ---

# (노트북 표준 세팅 코드에서 DATA_DIR이 정의되었다고 가정)
# DATA_DIR = os.path.join(PROJECT_ROOT, "data", "processed")
MASTER_TABLE_PATH = os.path.join(DATA_DIR, "final_master_table_v2.csv")

print("--- [Phase 2-A/C] DataHandler V2 초기화 ---")
data_handler = DataHandler(MASTER_TABLE_PATH, train_end_date='2022-12-31')

# 모델 하이퍼파라미터
INPUT_SEQ_LEN = 120  # 120일 (Input)
OUTPUT_SEQ_LEN = 10  # 10일 (Target)

if '010140' in data_handler.get_all_tickers():
    print("\n--- [Phase 2-B] '010140' 윈도우 생성 테스트 ---")
    
    # 1. 표준화된 2D 데이터 가져오기
    scaled_df = data_handler.get_scaled_data_by_ticker('010140')
    
    # 2. 3D 윈도우 생성
    X_train_samsung, y_train_samsung = create_sliding_windows(
        scaled_df, 
        INPUT_SEQ_LEN, 
        OUTPUT_SEQ_LEN
    )
    
    print("\n[SUCCESS] 3D Tensors (Windows) created!")
    print(f"  > X (Input Tensors) shape: {X_train_samsung.shape}")
    print(f"  > y (Target Tensors) shape: {y_train_samsung.shape}")
else:
    print("\n[Error] '010140' Ticker not found.")

In [12]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os

# --- 1. [Phase 2 실행] 데이터 로드 및 윈도우 생성 ---
# (커널 재시작했으면 이 부분을 꼭 다시 돌려야 변수가 생깁니다!)

# 경로 설정
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(PROJECT_ROOT, "data", "processed")
MASTER_TABLE_PATH = os.path.join(DATA_DIR, "final_master_table_v2.csv")

print("--- [Phase 2] 데이터 생성 및 윈도우 변환 시작 ---")

# 1. DataHandler 초기화
# (위쪽 셀에서 DataHandler 클래스가 정의되어 있어야 함)
if 'DataHandler' not in locals():
    print("❌ [ERROR] DataHandler 클래스가 정의되지 않았습니다. 맨 위쪽 클래스 정의 셀을 먼저 실행해주세요!")
else:
    data_handler = DataHandler(MASTER_TABLE_PATH, train_end_date='2022-12-31')

    # 2. 윈도우 생성
    INPUT_SEQ_LEN = 120
    OUTPUT_SEQ_LEN = 10

    if '010140' in data_handler.get_all_tickers():
        scaled_df = data_handler.get_scaled_data_by_ticker('010140')
        
        # 여기서 X_train_samsung 변수가 생성됩니다!
        X_train_samsung, y_train_samsung = create_sliding_windows(
            scaled_df, INPUT_SEQ_LEN, OUTPUT_SEQ_LEN
        )
        print(f"✅ [Phase 2 완료] 윈도우 생성됨: {X_train_samsung.shape}")
    else:
        print("❌ [ERROR] 010140 데이터 없음")

# --- 2. [Phase 3-B] 데이터셋 및 로더 정의 ---

class ShipDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# --- 3. [Phase 3-B] 실행: Train/Val/Test 분리 및 로더 생성 ---

if 'X_train_samsung' in locals():
    print("\n--- [Phase 3-B] 데이터 로더 생성 시작 ---")
    
    X_data = X_train_samsung
    y_data = y_train_samsung
    total_samples = len(X_data)
    
    # 7:1:2 분할
    train_size = int(total_samples * 0.7)
    val_size = int(total_samples * 0.1)
    
    X_train = X_data[:train_size]
    y_train = y_data[:train_size]
    
    X_val = X_data[train_size : train_size+val_size]
    y_val = y_data[train_size : train_size+val_size]
    
    X_test = X_data[train_size+val_size:]
    y_test = y_data[train_size+val_size:]
    
    # 데이터셋 생성
    train_dataset = ShipDataset(X_train, y_train)
    val_dataset = ShipDataset(X_val, y_val)
    test_dataset = ShipDataset(X_test, y_test)
    
    # 데이터 로더 생성 (Batch Size 32)
    BATCH_SIZE = 32
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    print(f"✅ [SUCCESS] 모든 준비 완료!")
    print(f"  > Train Loader: {len(train_loader)} batches")
    print(f"  > Val Loader  : {len(val_loader)} batches")
    print(f"  > Test Loader : {len(test_loader)} batches")
    
else:
    print("❌ [ERROR] 변수 생성 실패. 위 코드를 다시 확인해주세요.")

In [13]:
import torch
import os
import sys

# 1. 경로 설정 (혹시 모르니 다시)
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
GPT2_PATH = os.path.join(PROJECT_ROOT, "pretrained_models", "gpt2")
TIME_LLM_ROOT = os.path.join(PROJECT_ROOT, "external", "time-llm")

if TIME_LLM_ROOT not in sys.path:
    sys.path.append(TIME_LLM_ROOT)

# 2. Time-LLM 임포트
try:
    import models.TimeLLM
    import importlib
    importlib.reload(models.TimeLLM) # 수정사항 반영을 위해 강제 리로드
    from models.TimeLLM import Model as TimeLLM
except ImportError:
    print("❌ Time-LLM import failed")

# 3. [최종] Configs 클래스 (빠짐없이 다 넣음!)
class Configs:
    def __init__(self):
        # [기본 설정]
        self.task_name = 'long_term_forecast'
        self.is_training = 1
        self.model_id = 'Stock_Prediction'
        self.model = 'TimeLLM'
        
        # [데이터 차원]
        self.seq_len = 120
        self.label_len = 60
        self.pred_len = 10
        self.enc_in = 12
        self.dec_in = 12
        self.c_out = 12
        
        # [LLM 설정]
        self.llm_model = 'GPT2' 
        self.llm_model_path = GPT2_PATH 
        self.llm_dim = 768       
        self.llm_layers = 12     
        
        self.patch_len = 16
        self.stride = 8
        self.d_model = 32
        self.d_ff = 128
        self.n_heads = 8
        self.dropout = 0.1
        
        # [!!! 에러 해결사: 누락된 설정들 추가 !!!]
        self.prompt_domain = 1   # <--- 아까 그 에러의 원인!
        self.content = "Predict the future stock price based on financial indicators."
        
        # (혹시 모를 다음 에러 방지용 안전장치들)
        self.embed = 'timeF'     # 시간 임베딩 방식
        self.freq = 'd'          # 데이터 빈도 (Daily)
        self.factor = 1          # Attention Factor
        self.moving_avg = 25     # Moving Average Kernel size
        self.e_layers = 2        # Encoder Layers
        self.d_layers = 1        # Decoder Layers
        self.top_k = 5           # Top-K optimization

# 4. 초기화 실행
if 'TimeLLM' in locals():
    print(f"⏳ Initializing Time-LLM (Final Attempt)...")
    try:
        configs = Configs()
        model = TimeLLM(configs)
        
        if torch.cuda.is_available():
            model = model.cuda()
            
        print("\n🎉 [SUCCESS] Real Time-LLM initialized! (드디어 성공!)")
        print(f"  > LLM Backbone: GPT-2 (from {configs.llm_model_path})")
        print(f"  > Trainable Params: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
        
    except Exception as e:
        print(f"\n❌ [FAIL] Init Error: {e}")
        import traceback
        traceback.print_exc()

In [14]:
import torch
import torch.optim as optim
import torch.nn as nn

# ★ GPU 사용 설정 (가장 먼저 실행해야 함)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

from tqdm import tqdm
import importlib
import models.TimeLLM 
importlib.reload(models.TimeLLM) # 모델 구조 리로드
from models.TimeLLM import Model as TimeLLM

import sys
import os
from datetime import datetime

# === 로그 저장 경로 설정 ===
LOG_DIR = os.path.join(PROJECT_ROOT, "logs")
os.makedirs(LOG_DIR, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_PATH = os.path.join(LOG_DIR, f"train_log_{timestamp}.txt")

# === Logger 클래스: console + file 동시에 출력 ===
class Logger(object):
    def __init__(self, file_path):
        self.terminal = sys.stdout
        self.log = open(file_path, "a", encoding="utf-8")

    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)
        self.log.flush()  # 즉시 기록

    def flush(self):
        pass

# === sys.stdout 교체 ===
sys.stdout = Logger(LOG_PATH)
print(f"[LOGGING] Training logs will be saved to {LOG_PATH}")


# --- 1. [Ultimate] Configs 설정 ---
class Configs:
    def __init__(self):
        self.task_name = 'long_term_forecast'
        self.is_training = 1
        self.model_id = 'Stock_Prediction'
        self.model = 'TimeLLM'
        self.seq_len = 120
        self.label_len = 60
        self.pred_len = 10
        self.enc_in = 12
        self.dec_in = 12
        self.c_out = 12
        self.llm_model = 'GPT2' 
        self.llm_model_path = GPT2_PATH 
        self.llm_dim = 768       
        self.llm_layers = 6     
        
        # [전략 2: 현미경 모드] 
        # 16일씩 보던 걸 8일로 줄여서 더 디테일하게 봅니다.
        self.patch_len = 32      # (기존 16 -> 8)
        self.stride = 16          # (기존 8 -> 4: 더 촘촘하게 겹침)
        
        # [Monster Spec 유지]
        self.d_model = 256      
        self.d_ff = 256          
        self.n_heads = 12        
        self.dropout = 0.02      
        
        self.prompt_domain = 1
        
        # [전략 1: Rich PaP] 
        # 조선업 특화 정보를 아주 구체적으로 넣어줍니다.
        self.content = (
            "Task: Forecast daily closing prices for Korean Shipbuilding companies. "
            "Input Data: 12 channels including OHLC prices, Trading Volume, "
            "and Macro-indicators: Brent Oil Price, USD/KRW Exchange Rate, Interest Rate, and BDI (Baltic Dry Index). "
            "Context: Shipbuilding stocks are sensitive to Oil prices and BDI. "
            "Analyze the 120-day trend, focusing on volatility and correlation between macro-indicators and stock price. "
            "Predict the next 10 days."
        )

# 2. 모델 재초기화
configs = Configs()
model = TimeLLM(configs)
model.to(device).float()

print(f"🔥 Start Ultimate Training on {device}...")
print(f"   > Strategy: Rich Prompt + High Resolution (Patch 32)")

# 3. 학습 설정
# [전략 3: 진득하게 학습]
LEARNING_RATE = 0.00025  # (기존 0.0005 -> 0.0001: 더 세밀하게)
EPOCHS = 30             # (기존 15 -> 30: 더 오래)
ACCUM_STEPS = 8         # (Batch 4 * 8 = 32 효과)

optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
criterion = nn.MSELoss()

# 4. 학습 루프
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    optimizer.zero_grad()
    
    progress_bar = tqdm(train_loader_global, desc=f"Epoch {epoch+1}/{EPOCHS}")
    
    for i, (batch_x, batch_y) in enumerate(progress_bar):
        batch_x = batch_x.to(device).float()
        batch_y = batch_y.to(device).float()
        
        B, Seq, C = batch_x.shape
        Pred = batch_y.shape[1]
        
        dummy_mark_enc = torch.zeros(B, Seq, 4).to(device).float()
        dummy_mark_dec = torch.zeros(B, Pred, 4).to(device).float()
        dummy_dec_in = torch.zeros(B, Pred, C).to(device).float()
        
        outputs = model(batch_x, dummy_mark_enc, dummy_dec_in, dummy_mark_dec)
        if isinstance(outputs, tuple): outputs = outputs[0]
            
        f_dim = -1 if configs.c_out == 1 else 0
        preds = outputs[:, -configs.pred_len:, f_dim:]
        true = batch_y
        
        loss = criterion(preds, true)
        loss = loss / ACCUM_STEPS
        loss.backward()
        
        if (i + 1) % ACCUM_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()
            
        current_loss = loss.item() * ACCUM_STEPS
        total_loss += current_loss
        
        current_lr = optimizer.param_groups[0]['lr']
        progress_bar.set_postfix({'loss': f"{current_loss:.5f}", 'lr': f"{current_lr:.6f}"})
        
    scheduler.step()
    avg_loss = total_loss / len(train_loader_global)
    print(f"Epoch {epoch+1} Done. Avg Loss: {avg_loss:.5f}")

# 5. 최종 저장 (덮어쓰기)
SAVE_DIR = os.path.join(PROJECT_ROOT, "models")
os.makedirs(SAVE_DIR, exist_ok=True)
SAVE_PATH = os.path.join(SAVE_DIR, "ship_time_llm_tmp2.pth")
torch.save(model.state_dict(), SAVE_PATH)
print(f"\n💾 [ULTIMATE] Model Saved: {SAVE_PATH}")

NameError: name 'train_loader_global' is not defined

In [15]:
import matplotlib.pyplot as plt
import torch
import numpy as np

# ==========================================
# 1. 긴급! 모델과 데이터를 CPU로 피신시킵니다.
# ==========================================
# GPU 메모리가 꽉 찼으니 CPU로 모델을 옮깁니다.
model = model.cpu()
model.eval()

print("✅ Model moved to CPU for visualization.")

# 데이터 로더에서 하나 뽑기
data_iter = iter(train_loader_global)
batch_x, batch_y = next(data_iter)

# 데이터도 CPU에 둡니다 (굳이 GPU로 보낼 필요 없음)
batch_x = batch_x.cpu().float()
batch_y = batch_y.cpu().float()

# ==========================================
# 2. CPU에서 추론 (Inference)
# ==========================================
print("🤖 CPU에서 예측 수행 중... (메모리 걱정 NO)")

with torch.no_grad():
    # 가짜 입력 생성 (CPU용)
    B, Seq, C = batch_x.shape
    Pred = batch_y.shape[1]
    
    dummy_mark_enc = torch.zeros(B, Seq, 4).cpu().float()
    dummy_mark_dec = torch.zeros(B, Pred, 4).cpu().float()
    dummy_dec_in = torch.zeros(B, Pred, C).cpu().float()
    
    # 예측 실행
    outputs = model(batch_x, dummy_mark_enc, dummy_dec_in, dummy_mark_dec)
    
    if isinstance(outputs, tuple):
        outputs = outputs[0]

# ==========================================
# 3. 시각화
# ==========================================
sample_idx = 0
pred_seq = outputs[sample_idx, :, 0].numpy() # 이미 CPU라 .cpu() 불필요
true_seq = batch_y[sample_idx, :, 0].numpy()

plt.figure(figsize=(12, 6))
plt.plot(range(Pred), true_seq, label='Actual (Ground Truth)', 
         marker='o', color='black', linewidth=2, markersize=8)
plt.plot(range(Pred), pred_seq, label='AI Prediction (Forecast)', 
         marker='x', linestyle='--', color='red', linewidth=2, markersize=8)

plt.title(f"Ultimate Time-LLM Forecast (Loss: 0.4224) - Sample #{sample_idx}", fontsize=14, fontweight='bold')
plt.xlabel("Future Days (Day 1 to 10)")
plt.ylabel("Normalized Value")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

NameError: name 'train_loader_global' is not defined

In [12]:
import torch
import os
import sys
import numpy as np
from tqdm import tqdm
import importlib
import models.TimeLLM 

# 1. 기본 설정
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
GPT2_PATH = os.path.join(PROJECT_ROOT, "pretrained_models", "gpt2")
SAVE_PATH = os.path.join(PROJECT_ROOT, "models", "ship_time_llm_tmp2.pth")

# [GPU 설정] 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🔥 Using Device: {device} (RTX 5090 Power!)")

# 2. Configs (Ultimate 설정 동일)
class Configs:
    def __init__(self):
        self.task_name = 'long_term_forecast'
        self.is_training = 0
        self.model_id = 'Stock_Prediction'
        self.model = 'TimeLLM'
        self.seq_len = 120
        self.label_len = 60
        self.pred_len = 10
        self.enc_in = 12
        self.dec_in = 12
        self.c_out = 12
        self.llm_model = 'GPT2' 
        self.llm_model_path = GPT2_PATH 
        self.llm_dim = 768       
        self.llm_layers = 12     
        self.patch_len = 8   
        self.stride = 4      
        self.d_model = 768   
        self.d_ff = 768      
        self.n_heads = 12        
        self.dropout = 0.0
        self.prompt_domain = 1
        self.content = "Predict..."

# 3. 모델 구조 생성
importlib.reload(models.TimeLLM)
from models.TimeLLM import Model as TimeLLM

configs = Configs()
model = TimeLLM(configs)

# 4. [핵심] GPU로 모델 로드
if os.path.exists(SAVE_PATH):
    print(f"📂 Loading model from: {SAVE_PATH}")
    
    # (1) GPU 메모리로 직접 로드
    state_dict = torch.load(SAVE_PATH, map_location=device) 
    model.load_state_dict(state_dict)
    
    # (2) 모델을 GPU에 올림
    model.to(device).float()
    model.eval()
    
    print("✅ Model successfully loaded on GPU!")
else:
    print("❌ Model file not found.")
    raise SystemExit

# ==========================================================
# 5. 방향성 정확도(DA) 계산 (GPU 가속)
# ==========================================================
if 'train_loader_global' not in locals():
    print("❌ 'train_loader_global'이 없습니다. 위쪽 데이터 로더 복구 코드를 먼저 실행해주세요.")
else:
    print("\n📊 Calculating Directional Accuracy (DA) on GPU...")
    correct_direction = 0
    total_samples = 0

    with torch.no_grad():
        for batch_x, batch_y in tqdm(train_loader_global):
            # [수정] 데이터를 GPU로 이동
            batch_x = batch_x.to(device).float()
            batch_y = batch_y.to(device).float()
            
            B, Seq, C = batch_x.shape
            Pred = batch_y.shape[1]
            
            # [수정] 가짜 입력도 GPU로 생성
            dummy_mark_enc = torch.zeros(B, Seq, 4).to(device).float()
            dummy_mark_dec = torch.zeros(B, Pred, 4).to(device).float()
            dummy_dec_in = torch.zeros(B, Pred, C).to(device).float()
            
            # 추론
            outputs = model(batch_x, dummy_mark_enc, dummy_dec_in, dummy_mark_dec)
            if isinstance(outputs, tuple): outputs = outputs[0]
            
            # 결과 계산 (GPU 텐서를 CPU로 내려서 numpy 변환)
            current = batch_x[:, -1, 0].cpu().numpy()
            true_fut = batch_y[:, -1, 0].cpu().numpy()
            pred_fut = outputs[:, -1, 0].cpu().numpy()
            
            hits = ((true_fut - current) * (pred_fut - current)) > 0
            correct_direction += np.sum(hits)
            total_samples += B

    accuracy = (correct_direction / total_samples) * 100
    print(f"\n🎯 Final DA (GPU Verified): {accuracy:.2f}%")

🔥 Using Device: cuda (RTX 5090 Power!)
📂 Loading model from: /workspace/ship-ai/models/ship_time_llm_tmp2.pth
✅ Model successfully loaded on GPU!

📊 Calculating Directional Accuracy (DA) on GPU...


100%|█████████████████████████████████████████| 143/143 [01:00<00:00,  2.37it/s]


🎯 Final DA (GPU Verified): 55.22%


In [13]:
# 🔬 [심화 검증] 확신 필터링 (Confidence Filtering) 승률 계산

print("📊 High Confidence 승률 분석 중...")

total_trades = 0
winning_trades = 0
threshold = 0.02  # "2% 이상 오르거나 내린다고 할 때만 진입하겠다" (기준점)

with torch.no_grad():
    for batch_x, batch_y in train_loader_global:
        batch_x = batch_x.to(device).float()
        batch_y = batch_y.to(device).float()
        
        # ... (추론 코드 동일) ...
        B, Seq, C = batch_x.shape
        Pred = batch_y.shape[1]
        dummy_mark_enc = torch.zeros(B, Seq, 4).to(device).float()
        dummy_mark_dec = torch.zeros(B, Pred, 4).to(device).float()
        dummy_dec_in = torch.zeros(B, Pred, C).to(device).float()
        
        outputs = model(batch_x, dummy_mark_enc, dummy_dec_in, dummy_mark_dec)
        if isinstance(outputs, tuple): outputs = outputs[0]
        
        # 값 추출
        current = batch_x[:, -1, 0].cpu().numpy()
        true_fut = batch_y[:, -1, 0].cpu().numpy()
        pred_fut = outputs[:, -1, 0].cpu().numpy()
        
        # [핵심 로직] 예측 변동폭 계산
        pred_change_pct = (pred_fut - current) / current
        
        # 2% 이상 급등/급락 예측한 경우만 카운트 (필터링)
        mask = np.abs(pred_change_pct) >= threshold 
        
        if np.sum(mask) > 0:
            true_chg = true_fut[mask] - current[mask]
            pred_chg = pred_fut[mask] - current[mask]
            
            # 방향 맞춤?
            hits = (true_chg * pred_chg) > 0
            
            winning_trades += np.sum(hits)
            total_trades += np.sum(mask)

if total_trades > 0:
    win_rate = (winning_trades / total_trades) * 100
    print("-" * 40)
    print(f"🎯 [필터링 적용 후] 승률 분석 (기준: 변동폭 {threshold*100}%)")
    print(f"   > 진입 횟수: {total_trades}회 (전체의 약 {total_trades/total_samples*100:.1f}%)")
    print(f"   > 승리 횟수: {winning_trades}회")
    print(f"   > 필터링 승률: {win_rate:.2f}% 🔥")
    print("-" * 40)
else:
    print("⚠️ 해당 기준만큼 강하게 예측한 샘플이 없습니다.")

📊 High Confidence 승률 분석 중...
----------------------------------------
🎯 [필터링 적용 후] 승률 분석 (기준: 변동폭 2.0%)
   > 진입 횟수: 4144회 (전체의 약 90.7%)
   > 승리 횟수: 2302회
   > 필터링 승률: 55.55% 🔥
----------------------------------------


In [14]:
import numpy as np
import torch

# 모델을 GPU로 이동 (빠른 계산)
model.to(device)
model.eval()

thresholds = [0.03, 0.05, 0.07, 0.10] # 3%, 5%, 7%, 10% 기준 테스트

print("📊 [Sniper Test] 기준별 승률 및 수익성 분석")
print("="*65)
print(f"{'기준(Threshold)':<15} | {'진입(%)':<12} | {'승률(Win%)':<12} | {'평균수익(Avg%)':<15}")
print("-" * 65)

with torch.no_grad():
    # 전체 데이터를 미리 리스트로 모음 (속도 최적화)
    all_current = []
    all_true_fut = []
    all_pred_fut = []
    
    for batch_x, batch_y in train_loader_global:
        batch_x = batch_x.to(device).float()
        batch_y = batch_y.to(device).float()
        
        B, Seq, C = batch_x.shape
        Pred = batch_y.shape[1]
        dummy_mark_enc = torch.zeros(B, Seq, 4).to(device).float()
        dummy_mark_dec = torch.zeros(B, Pred, 4).to(device).float()
        dummy_dec_in = torch.zeros(B, Pred, C).to(device).float()
        
        outputs = model(batch_x, dummy_mark_enc, dummy_dec_in, dummy_mark_dec)
        if isinstance(outputs, tuple): outputs = outputs[0]
        
        # CPU로 이동 후 저장
        all_current.append(batch_x[:, -1, 0].cpu().numpy())
        all_true_fut.append(batch_y[:, -1, 0].cpu().numpy())
        all_pred_fut.append(outputs[:, -1, 0].cpu().numpy())

    # Numpy 배열로 변환
    all_current = np.concatenate(all_current)
    all_true_fut = np.concatenate(all_true_fut)
    all_pred_fut = np.concatenate(all_pred_fut)
    
    # --- 기준별 테스트 루프 ---
    for th in thresholds:
        # 예측 변동률 계산
        pred_change_pct = (all_pred_fut - all_current) / all_current
        
        # 필터링 (절대값 기준)
        mask = np.abs(pred_change_pct) >= th
        
        total_trades = np.sum(mask)
        
        if total_trades > 0:
            # 실제 변동률
            true_change_pct = (all_true_fut[mask] - all_current[mask]) / all_current[mask]
            # 예측 변동률 (필터링 된 것만)
            pred_change_pct_filtered = pred_change_pct[mask]
            
            # 1. 승률 (방향 일치)
            hits = (true_change_pct * pred_change_pct_filtered) > 0
            win_rate = np.mean(hits) * 100
            
            # 2. 수익성 (방향 맞으면 수익, 틀리면 손실)
            # (Long/Short 전략 가정: 오를 거라 예측하면 매수, 내릴 거라 예측하면 매도)
            # 수익 = 예측방향 * 실제변동
            # 예: 예측(+), 실제(+) -> 수익 / 예측(+), 실제(-) -> 손실
            returns = np.sign(pred_change_pct_filtered) * true_change_pct
            avg_return = np.mean(returns) * 100
            
            participation = (total_trades / len(all_current)) * 100
            
            print(f"{th*100:>4.0f}% 이상 변동 | {participation:>5.1f}% ({total_trades}) | {win_rate:>6.2f}% 🔥   | {avg_return:>+6.2f}% 💰")
        else:
            print(f"{th*100:>4.0f}% 이상 변동 |   0.0% (0)    |    -         |    -")

print("="*65)

📊 [Sniper Test] 기준별 승률 및 수익성 분석
기준(Threshold)   | 진입(%)        | 승률(Win%)     | 평균수익(Avg%)     
-----------------------------------------------------------------
   3% 이상 변동 |  86.9% (3973) |  55.83% 🔥   | +1122.66% 💰
   5% 이상 변동 |  78.4% (3585) |  56.15% 🔥   | +1244.16% 💰
   7% 이상 변동 |  70.1% (3202) |  56.90% 🔥   | +1393.22% 💰
  10% 이상 변동 |  59.9% (2740) |  57.85% 🔥   | +1627.50% 💰


In [26]:
import torch
import numpy as np
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
def eval_loader(loader, name="train"):
    model.eval()

    mse_model_list = []
    mse_naive_list = []
    dir_acc_model_list = []
    dir_acc_naive_list = []

    with torch.no_grad():
        for batch_x, batch_y in loader:
            batch_x = batch_x.to(device).float()      # (B, Seq, C)
            batch_y = batch_y.to(device).float()      # (B, Pred, C)

            B, Seq, C = batch_x.shape
            Pred = batch_y.shape[1]

            dummy_mark_enc = torch.zeros(B, Seq, 4).to(device)
            dummy_mark_dec = torch.zeros(B, Pred, 4).to(device)
            dummy_dec_in   = torch.zeros(B, Pred, C).to(device)

            outputs = model(batch_x, dummy_mark_enc, dummy_dec_in, dummy_mark_dec)
            if isinstance(outputs, tuple):
                outputs = outputs[0]

            # --- 모델 예측 (0번 채널 기준: 종가라고 가정) ---
            f_dim = -1 if configs.c_out == 1 else 0
            preds = outputs[:, -configs.pred_len:, f_dim:]    # (B, Pred, C)

            true = batch_y[:, :, 0]    # (B, Pred)
            pred = preds[:, :, 0]      # (B, Pred)

            # ===== 1) MSE 비교 =====
            mse_model = torch.mean((pred - true) ** 2).item()

            # naive: 마지막 입력값을 그대로 10일 복사 (레벨 기준 baseline)
            naive = batch_x[:, -1, 0].unsqueeze(1).repeat(1, Pred)  # (B, Pred)
            mse_naive = torch.mean((naive - true) ** 2).item()

            mse_model_list.append(mse_model)
            mse_naive_list.append(mse_naive)

            # ===== 2) 방향 정확도(DIR%) 비교 =====
            true_ret = true[:, 1:] - true[:, :-1]     # (B, Pred-1)
            pred_ret = pred[:, 1:] - pred[:, :-1]

            true_sign = torch.sign(true_ret)
            pred_sign = torch.sign(pred_ret)

            # naive 방향: 마지막 입력 구간의 방향을 그대로 유지
            last_hist_ret = batch_x[:, -1, 0] - batch_x[:, -2, 0]   # (B,)
            naive_sign = torch.sign(last_hist_ret).unsqueeze(1).repeat(1, Pred-1)

            # true 변화가 0인 구간은 제외
            mask = true_sign != 0
            if mask.sum() == 0:
                continue

            model_correct = (pred_sign[mask] == true_sign[mask]).float().mean().item()
            naive_correct = (naive_sign[mask] == true_sign[mask]).float().mean().item()

            dir_acc_model_list.append(model_correct)
            dir_acc_naive_list.append(naive_correct)
            

    avg_mse_model = np.mean(mse_model_list)
    avg_mse_naive = np.mean(mse_naive_list)
    avg_dir_model = np.mean(dir_acc_model_list) if len(dir_acc_model_list) > 0 else float('nan')
    avg_dir_naive = np.mean(dir_acc_naive_list) if len(dir_acc_naive_list) > 0 else float('nan')

    print(f"[{name}] MSE   model={avg_mse_model:.4f}, naive={avg_mse_naive:.4f}")
    print(f"[{name}] DIR%  model={avg_dir_model*100:.2f}%, naive={avg_dir_naive*100:.2f}%")
    print("-" * 60)


# 🔹 실제 실행
eval_loader(train_loader, "train")
eval_loader(val_loader,   "val")


KeyboardInterrupt: 

In [28]:
# 1) 입력 시퀀스 마지막 값 (scaled)
print("X_last_scaled:", batch_x[0, -1, 0].item())

# 2) 정답 시퀀스 평균값 (scaled)
print("Y_mean_scaled:", batch_y[0, :, 0].mean().item())

# 3) 예측값 평균 (scaled)
print("Pred_mean_scaled:", pred[0].mean().item())


X_last_scaled: 0.9607056379318237
Y_mean_scaled: 0.7976146936416626
Pred_mean_scaled: 3.3096923828125


In [32]:
true_ret = true[:, -1] - true[:, 0]
pred_ret = pred[:, -1] - pred[:, 0]

ret_mse = torch.mean((true_ret - pred_ret)**2).item()
print(ret_mse)

0.18678079545497894


In [33]:
import torch
import numpy as np

def eval_horizon(loader, name="val"):
    model.to(device)
    model.eval()

    Pred = configs.pred_len

    mse_model_h = [[] for _ in range(Pred)]
    mse_naive_h = [[] for _ in range(Pred)]
    dir_model_h = [[] for _ in range(Pred-1)]
    dir_naive_h = [[] for _ in range(Pred-1)]

    with torch.no_grad():
        for batch_x, batch_y in loader:
            batch_x = batch_x.to(device).float()   # (B, Seq, C)
            batch_y = batch_y.to(device).float()   # (B, Pred, C)

            B, Seq, C = batch_x.shape
            Pred = batch_y.shape[1]

            dummy_mark_enc = torch.zeros(B, Seq, 4, device=device)
            dummy_mark_dec = torch.zeros(B, Pred, 4, device=device)
            dummy_dec_in   = torch.zeros(B, Pred, C, device=device)

            outputs = model(batch_x, dummy_mark_enc, dummy_dec_in, dummy_mark_dec)
            if isinstance(outputs, tuple):
                outputs = outputs[0]

            f_dim = -1 if configs.c_out == 1 else 0
            preds = outputs[:, -configs.pred_len:, f_dim:]   # (B, Pred, C)

            true = batch_y[:, :, 0]   # (B, Pred)
            pred = preds[:, :, 0]     # (B, Pred)

            # naive: 마지막 입력값 고정
            naive = batch_x[:, -1, 0].unsqueeze(1).repeat(1, Pred)  # (B, Pred)

            # --- 1) Horizon별 MSE ---
            for h in range(Pred):
                mse_m = torch.mean((pred[:, h] - true[:, h])**2).item()
                mse_n = torch.mean((naive[:, h] - true[:, h])**2).item()
                mse_model_h[h].append(mse_m)
                mse_naive_h[h].append(mse_n)

            # --- 2) Horizon별 방향 정확도 ---
            true_ret  = true[:, 1:] - true[:, :-1]   # (B, Pred-1)
            pred_ret  = pred[:, 1:] - pred[:, :-1]
            naive_ret = naive[:, 1:] - naive[:, :-1]

            true_sign  = torch.sign(true_ret)
            pred_sign  = torch.sign(pred_ret)

            last_hist_ret = batch_x[:, -1, 0] - batch_x[:, -2, 0]
            naive_sign = torch.sign(last_hist_ret).unsqueeze(1).repeat(1, Pred-1)

            for h in range(Pred-1):
                ts = true_sign[:, h]
                ps = pred_sign[:, h]
                ns = naive_sign[:, h]

                mask = ts != 0
                if mask.sum() == 0:
                    continue

                acc_m = (ps[mask] == ts[mask]).float().mean().item()
                acc_n = (ns[mask] == ts[mask]).float().mean().item()

                dir_model_h[h].append(acc_m)
                dir_naive_h[h].append(acc_n)

    # 평균 취하기
    mse_model_h = [np.mean(v) if len(v) > 0 else np.nan for v in mse_model_h]
    mse_naive_h = [np.mean(v) if len(v) > 0 else np.nan for v in mse_naive_h]
    dir_model_h = [np.mean(v)*100 if len(v) > 0 else np.nan for v in dir_model_h]
    dir_naive_h = [np.mean(v)*100 if len(v) > 0 else np.nan for v in dir_naive_h]

    print(f"=== [{name}] Horizon별 MSE (h=0은 +1일차) ===")
    for h in range(Pred):
        print(f"h+{h+1}: MSE_model={mse_model_h[h]:.4f}, MSE_naive={mse_naive_h[h]:.4f}")

    print(f"\n=== [{name}] Horizon별 방향 정확도 (1~9일 구간) ===")
    for h in range(Pred-1):
        print(f"구간 {h+1}->{h+2}: DIR_model={dir_model_h[h]:.2f}%, DIR_naive={dir_naive_h[h]:.2f}%")

    return mse_model_h, mse_naive_h, dir_model_h, dir_naive_h


# 실제 실행 (val 기준)
eval_horizon(val_loader, "val")


=== [val] Horizon별 MSE (h=0은 +1일차) ===
h+1: MSE_model=0.0547, MSE_naive=0.0247
h+2: MSE_model=0.0824, MSE_naive=0.0538
h+3: MSE_model=0.1192, MSE_naive=0.0839
h+4: MSE_model=0.1401, MSE_naive=0.1136
h+5: MSE_model=0.1680, MSE_naive=0.1353
h+6: MSE_model=0.2096, MSE_naive=0.1596
h+7: MSE_model=0.2344, MSE_naive=0.1839
h+8: MSE_model=0.2915, MSE_naive=0.2140
h+9: MSE_model=0.3225, MSE_naive=0.2456
h+10: MSE_model=0.3631, MSE_naive=0.2776

=== [val] Horizon별 방향 정확도 (1~9일 구간) ===
구간 1->2: DIR_model=42.60%, DIR_naive=55.70%
구간 2->3: DIR_model=55.90%, DIR_naive=41.68%
구간 3->4: DIR_model=56.67%, DIR_naive=53.23%
구간 4->5: DIR_model=54.55%, DIR_naive=43.29%
구간 5->6: DIR_model=52.70%, DIR_naive=44.29%
구간 6->7: DIR_model=54.79%, DIR_naive=45.02%
구간 7->8: DIR_model=51.03%, DIR_naive=50.14%
구간 8->9: DIR_model=60.23%, DIR_naive=49.92%
구간 9->10: DIR_model=49.06%, DIR_naive=45.70%


([np.float64(0.05470380745828152),
  np.float64(0.08240102417767048),
  np.float64(0.11920730024576187),
  np.float64(0.14014775305986404),
  np.float64(0.168045561760664),
  np.float64(0.20955790579319),
  np.float64(0.23438460379838943),
  np.float64(0.29151125252246857),
  np.float64(0.3225190341472626),
  np.float64(0.3630925267934799)],
 [np.float64(0.024657076224684715),
  np.float64(0.05379923526197672),
  np.float64(0.08392051979899406),
  np.float64(0.11361845582723618),
  np.float64(0.13533006608486176),
  np.float64(0.15955558605492115),
  np.float64(0.1839192993938923),
  np.float64(0.21397481113672256),
  np.float64(0.2455967739224434),
  np.float64(0.27761025726795197)],
 [np.float64(42.59877875447273),
  np.float64(55.89619427919388),
  np.float64(56.67205452919006),
  np.float64(54.54741418361664),
  np.float64(52.70114988088608),
  np.float64(54.79166731238365),
  np.float64(51.02909505367279),
  np.float64(60.22988557815552),
  np.float64(49.06250089406967)],
 [np.flo